In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys, os
sys.path.append("../subjects/")
import time, random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from fairlearn.metrics import equalized_odds_difference
from Search_utils import causal_dataset_student
from sklearn.feature_selection import SelectKBest, SelectFpr,SelectPercentile 

import matplotlib.pyplot as plt
import itertools
org_data = pd.read_csv('../subjects/datasets/student_PP')
coef_data = pd.read_csv('../Results/LogisticRegression/student/sex/student_coef.csv')

#coef_data['ae'] =coef_data['ae'] +0.5

    
#coef_data['ry']=coef_data['ry'] +4.2
coef_list  = coef_data.columns[8:]
    
coef_mean = coef_data[coef_data.columns[8:]].mean()
non_sensitive = ['failures', 'higher', 'G1', 'G2']


sensitive_param = 'sex'

transformer ='SelectKBest'#,'SelectFpr','SelectPercentile']:
feat_name_list = []
#coef_data['sy']=coef_data['sy'] -2.5

analysis = 'sensitive'
drop_analysis_result_SelectKBest=[]
drop_analysis_result_SelectFpr=[]
drop_analysis_result_SelectPercentile=[]
drop_analysis_result_sens= []
stp_range =[i/10 for i in range(-2,2)]
results ={}
for coef in coef_list:
    #coef_data[coef] =7
    if '0' in coef or 'sigma' in coef:continue
        
    for step_size in [-2]:#stp_range: 
        if step_size==0:
            continue

        eod_d =[]
        acc_d =[]
        f1_d=[]

        for exp in range(10):
            status, df, bal, EOD_None, acc_None, f1_None = causal_dataset_student(org_data, coef_data,
                                                        coef_list, coef, step_size  ,False, sensitive_param , default_acc=None  , 
                                                        default_f1=None, random_state=None,  drop_feat=None)
            #print(EOD_None)
            if status==False:
                continue
            
            A_new = df[sensitive_param].to_numpy()
            X_new = df[df.columns[:-1]].drop(columns=[sensitive_param])
            Y_new = df[df.columns[-1]]
            if transformer == 'SelectKBest' and analysis == 'nonsensitive':
                Kbest = SelectKBest( k=len(non_sensitive)-1)
                X_new = Kbest.fit_transform(X_new, Y_new)

            elif transformer == 'SelectFpr'and analysis == 'nonsensitive':
                sfpr =  SelectFpr(alpha=0.01)
                X_new = sfpr.fit_transform(X_new, Y_new)

            elif transformer == 'SelectPercentile'and analysis == 'nonsensitive':
                percentile =  SelectPercentile(percentile=10)
                X_new = percentile.fit_transform(X_new, Y_new)

            if analysis == 'nonsensitive':
                X_new = np.hstack((X_new,A_new.reshape(-1,1)))







            X_train_new, X_test_new, y_train_new, y_test_new, A_train_new, A_test_new = train_test_split(X_new, 
                                                                                                         Y_new,
                                                                                                         A_new, 
                                                                                                       random_state=0,
                                                                                                         stratify=Y_new)

            model = LogisticRegression()
            model.fit(X_train_new,y_train_new)
            preds_new = model.predict(X_test_new)
            acc_temp = accuracy_score(y_test_new,preds_new)
            f1_temp = f1_score(y_test_new,preds_new)
            EOD = round(equalized_odds_difference(y_test_new, preds_new, sensitive_features=A_test_new ),3)
            #print(EOD,EOD_None)
            acc_diff = round(acc_temp - acc_None,2)
            f1_diff = round(f1_temp- f1_None,2)
            EOD_diff = EOD - EOD_None
            #print(EOD_None, EOD)
            eod_d.append(EOD_diff)
            acc_d.append(acc_diff)
            f1_d.append(f1_diff)
            #print(coef, step_size,np.mean(eod_d),np.mean(acc_d),np.mean(f1_d))
        if transformer == 'SelectKBest' and analysis == 'nonsensitive':
            drop_analysis_result_SelectKBest.append([coef, step_size, eod_d, acc_d, f1_d])
        elif transformer == 'SelectFpr' and analysis == 'nonsensitive':
            drop_analysis_result_SelectFpr.append([coef, step_size, eod_d, acc_d, f1_d])
        elif transformer == 'SelectPercentile' and analysis == 'nonsensitive':
            drop_analysis_result_SelectPercentile.append([coef, step_size, eod_d, acc_d, f1_d])

        if analysis == 'sensitive':
            drop_analysis_result_sens.append([coef, step_size, eod_d, acc_d, f1_d])
        print(coef, step_size,np.mean(eod))
        if np.mean(eod_d)> 0.05 or np.mean(eod_d)<-0.05:
            print(coef, step_size,np.mean(eod_d),np.mean(acc_d),np.mean(f1_d))
# if transformer == 'SelectKBest':
#         np.save('../Results/LogisticRegression/redcar/st_test/RQ2/SelectKBest.npy',
#                   drop_analysis_result_SelectKBest)
# elif transformer == 'SelectFpr':
#             np.save('../Results/LogisticRegression/redcar/st_test/RQ2/SelectFpr.npy',
#                   drop_analysis_result_SelectFpr)
# elif transformer == 'SelectPercentile':
#             np.save('../Results/LogisticRegression/redcar/st_test/RQ2/SelectPercentile.npy',
#                   drop_analysis_result_SelectPercentile)
# if analysis == 'sensitive': 
#            np.save('../Results/LogisticRegression/student/st_test/RQ2/Sensitive.npy',
#                   drop_analysis_result_sens)

In [ ]:
if np.where(Y_new==0)[0].shape[0]<3: